<a href="https://colab.research.google.com/github/7oda111/study4fun/blob/main/detecting_movements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2 

In [3]:
import colorsys

In [6]:
mask=cv2.inRange(hsv,lower,upper)

NameError: ignored